In [1]:
# This is necessary to recognize the modules
import os
import sys
from decimal import Decimal
import warnings

warnings.filterwarnings("ignore")

root_path = os.path.abspath(os.path.join(os.getcwd(), '../..'))
sys.path.append(root_path)

In [2]:
from core.data_sources.clob import CLOBDataSource

# Get trading rules and candles
clob = CLOBDataSource()

2024-09-24 02:20:37,284 - asyncio - ERROR - Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x107244790>


In [3]:
# Constants
CONNECTOR_NAME = "okx"
INTERVAL = "1d"
DAYS = 360
FETCH_CANDLES = False
# Features configuration
VOLATILITY_WINDOW = 20
VOLUME_SHORT_WINDOW = 20
VOLUME_LONG_WINDOW = 50

# Download data
- Get trading rules
- Get candles for the last x days

In [4]:
import asyncio

trading_rules = await clob.get_trading_rules(CONNECTOR_NAME)
trading_pairs = trading_rules.filter_by_quote_asset("USDT")\
    .filter_by_min_notional_size(Decimal("5"))\
    .get_all_trading_pairs()

2024-09-24 02:20:37,905 - asyncio - ERROR - Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x14e973fa0>
2024-09-24 02:20:37,906 - asyncio - ERROR - Unclosed connector
connections: ['[(<aiohttp.client_proto.ResponseHandler object at 0x1071f1900>, 571680.458916625)]']
connector: <aiohttp.connector.TCPConnector object at 0x14e9734c0>


In [5]:
trading_pairs

['BONE-USDT',
 'PRQ-USDT',
 'RAY-USDT',
 'GLMR-USDT',
 'AVIVE-USDT',
 'CVX-USDT',
 'GMT-USDT',
 'LPT-USDT',
 'AUCTION-USDT',
 'ZRX-USDT',
 'MXC-USDT',
 'NMR-USDT',
 'AGLD-USDT',
 'LRC-USDT',
 'IQ-USDT',
 'MAGIC-USDT',
 'GFT-USDT',
 'UTK-USDT',
 'ICX-USDT',
 'DOGE-USDT',
 'ENJ-USDT',
 'YFI-USDT',
 'MLN-USDT',
 'KAN-USDT',
 'PRCL-USDT',
 'VENOM-USDT',
 'CFX-USDT',
 'KDA-USDT',
 'POLYDOGE-USDT',
 'DIA-USDT',
 'TNSR-USDT',
 'APT-USDT',
 'EGLD-USDT',
 'ETHW-USDT',
 'OKB-USDT',
 'WLD-USDT',
 'SOL-USDT',
 'CSPR-USDT',
 'ICP-USDT',
 'MILO-USDT',
 'INJ-USDT',
 'SAFE-USDT',
 'CHZ-USDT',
 'BNT-USDT',
 'SSWP-USDT',
 'BICO-USDT',
 'GODS-USDT',
 'LEO-USDT',
 'ARB-USDT',
 'AEVO-USDT',
 'METIS-USDT',
 'ZENT-USDT',
 'MERL-USDT',
 'G-USDT',
 'SAND-USDT',
 'FLR-USDT',
 'OM-USDT',
 'LQTY-USDT',
 'GMX-USDT',
 'LUNC-USDT',
 'VELODROME-USDT',
 'AVAX-USDT',
 'ETH-USDT',
 'BLUR-USDT',
 'OXT-USDT',
 'DAO-USDT',
 'IOST-USDT',
 'SAMO-USDT',
 'NEAR-USDT',
 'XR-USDT',
 'THETA-USDT',
 'REN-USDT',
 'CITY-USDT',
 'API

In [6]:
BATCH_CANDLES_REQUEST = 2
SLEEP_REQUEST = 2.0


if FETCH_CANDLES:
    candles = await clob.get_candles_batch_last_days(CONNECTOR_NAME, trading_pairs, INTERVAL, DAYS, BATCH_CANDLES_REQUEST, SLEEP_REQUEST)
    clob.dump_candles_cache(root_path)
else:
    clob.load_candles_cache(root_path)

In [7]:
candles = [value for key, value in clob.candles_cache.items() if key[2] == INTERVAL and key[0] == CONNECTOR_NAME]

In [8]:
from core.features.candles.volatility import VolatilityConfig
from core.features.candles.volume import VolumeConfig
from research_notebooks.dneitor.utils import generate_report

report = generate_report(
    candles=candles,
    volatility_config=VolatilityConfig(window=VOLATILITY_WINDOW),
    volume_config=VolumeConfig(short_window=VOLUME_SHORT_WINDOW, long_window=VOLUME_LONG_WINDOW))
report

Error processing POL-USDT: unsupported operand type(s) for /: 'NoneType' and 'int'
Error processing CATI-USDT: unsupported operand type(s) for /: 'NoneType' and 'int'


,trading_pair,mean_volatility,mean_natr,mean_bb_width,latest_trend,average_volume_per_hour,current_position,range_price_pct,normalized_score
0,MILO-USDT,0.12015105,0.14084198,57.17151534,-1,29938.48677793,0.76098522,0.94169284,0.01156202
1,NEO-USDT,0.04614496,0.0769013,29.93791474,-1,119192.86431087,0.77373305,0.69995912,0.01899478
2,GFT-USDT,0.06372236,0.1221588,36.84857235,-1,89156.63676779,0.89426758,0.7264929,0.02707471
3,SLP-USDT,0.05591378,0.09294876,36.59973258,-1,34022.14417855,0.81904311,0.82589984,0.00818514
4,RIO-USDT,0.07202075,0.12035815,50.97068945,-1,64899.83817987,0.57451472,0.95329055,0.01636937
...,...,...,...,...,...,...,...,...,...
289,ORB-USDT,0.07277634,0.13558808,49.00954413,-1,11531.66331575,0.95173018,0.88901869,0.00506203
290,ARKM-USDT,0.0458201,0.08115417,34.81660191,-1,21365.31978145,0.37974684,0.29743976,0.00074938
291,NOT-USDT,0.09016941,0.128562,58.75293163,-1,2129783.13495673,0.86079973,0.78727608,0.71000864
292,GALFT-USDT,0.05048451,0.09059968,29.93586053,-1,4140.36827405,0.88219895,0.70930472,0.00089816


In [9]:
from research_notebooks.dneitor.utils import filter_top_markets

TOP_X_MARKETS = 10  # Number of top markets to select
MIN_VOLUME_USD = 2000  # Minimum volume in USD
MIN_NATR = 0.01  # Minimum ATR
CURRENT_POSITION_THRESHOLD = 0.8  # Trend threshold

top_markets = filter_top_markets(report_df=report, top_x=TOP_X_MARKETS, min_volume_usd=MIN_VOLUME_USD, min_natr=MIN_NATR, current_position_threshold=CURRENT_POSITION_THRESHOLD)
top_markets_candles = {candle.trading_pair: candle for candle in candles if candle.trading_pair in top_markets["trading_pair"].values}
top_markets

,trading_pair,mean_volatility,mean_natr,mean_bb_width,latest_trend,average_volume_per_hour,current_position,range_price_pct,normalized_score
291,NOT-USDT,0.09016941,0.128562,58.75293163,-1,2129783.13495673,0.86079973,0.78727608,0.71000864
242,WLD-USDT,0.07444602,0.11919296,50.06082628,-1,1048887.32515181,0.97808669,0.88576315,0.41443921
226,DOGS-USDT,0.06476049,0.13812272,34.2198502,-1,1430342.92449739,1,0.43500339,0.3288398
156,FIL-USDT,0.0473841,0.07938209,31.59597349,-1,1407354.30374117,0.93361548,0.72264942,0.28840853
59,ETHFI-USDT,0.07050695,0.12593389,48.48634452,-1,681332.5215788,0.96465061,0.8448743,0.26757796
200,BIGTIME-USDT,0.08403233,0.14097898,56.82757786,-1,559363.79684727,0.92047862,0.92455568,0.25679227
154,STRK-USDT,0.05590467,0.10233372,38.36375113,-1,696749.18937831,0.98116678,0.86840103,0.23245818
254,PYTH-USDT,0.05977047,0.10790244,39.18555602,-1,773400.12266918,0.91955504,0.7762225,0.22792182
25,ULTI-USDT,0.09892,0.17359534,63.010477,-1,429208.38984384,0.95121951,0.75388967,0.20444724
161,ARB-USDT,0.04540451,0.07859069,31.07679454,-1,812890.68853631,0.93295084,0.78712477,0.17951139


In [13]:
from hummingbot.core.data_type.common import OrderType
from research_notebooks.dneitor.utils import generate_config
from core.utils import dump_dict_to_yaml


TOTAL_AMOUNT = 500  # General total amount for all markets
ACTIVATION_BOUNDS = 0.002  # Input activation bounds
MAX_OPEN_ORDERS = 1  # Input max open orders for each market
MIN_ORDER_AMOUNT = 2  # Minimum order amount for each market
LEVERAGE = 1  # Leverage for each market
TAKE_PROFIT_ORDER_TYPE = OrderType.MARKET

TAKE_PROFIT_MULTIPLIER = 0.2  # Multiplier for take profit based on NATR
AMOUNTS_QUOTE_PCT = [0.1, 0.1, 0.1, 0.1, 0.2, 0.2]  # Weights for each cluster

strategy_config = generate_config(
    id=f"dneitor-{CONNECTOR_NAME}_0.6",
    connector_name=CONNECTOR_NAME,
    candles=top_markets_candles,
    top_markets=top_markets,
    total_amount=TOTAL_AMOUNT,
    amounts_quote_pct=AMOUNTS_QUOTE_PCT,
    activation_bounds=ACTIVATION_BOUNDS,
    take_profit_multiplier=TAKE_PROFIT_MULTIPLIER,
    max_open_orders=MAX_OPEN_ORDERS,
    min_order_amount=MIN_ORDER_AMOUNT,
    leverage=LEVERAGE,
    take_profit_order_type=TAKE_PROFIT_ORDER_TYPE
)

In [14]:
strategy_config

[{'id': 'dneitor-okx_0.6_NOT-USDT',
  'controller_name': 'dneitor',
  'controller_type': 'generic',
  'manual_kill_switch': None,
  'candles_config': [],
  'connector_name': 'okx',
  'trading_pair': 'NOT-USDT',
  'total_amount_quote': 500,
  'prices': [0.022776,
   0.014573333333333334,
   0.0127005,
   0.008808,
   0.0069595,
   0.0045],
  'amounts_quote_pct': [0.1, 0.1, 0.1, 0.1, 0.2, 0.2],
  'take_profit': 0.025712400625299434,
  'activation_bounds': 0.002,
  'max_open_orders': 1,
  'min_order_amount': 1,
  'leverage': 1,
  'time_limit': None},
 {'id': 'dneitor-okx_0.6_WLD-USDT',
  'controller_name': 'dneitor',
  'controller_type': 'generic',
  'manual_kill_switch': None,
  'candles_config': [],
  'connector_name': 'okx',
  'trading_pair': 'WLD-USDT',
  'total_amount_quote': 500,
  'prices': [11.695,
   8.8,
   6.853666666666666,
   4.7382,
   3.575333333333333,
   1.856869230769231],
  'amounts_quote_pct': [0.1, 0.1, 0.1, 0.1, 0.2, 0.2],
  'take_profit': 0.023838591827325006,
  'ac

In [15]:
for config in strategy_config:
    dump_dict_to_yaml(config, "configs")